In [1]:
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

import mongo as mn
import foursquare as fs

In [2]:
load_dotenv()

c = mn.connectToMongo()
result = mn.getCompaniesAndLocation(c, 'games_video')
companies = mn.unflatten_result(result)
base_df = mn.buildBaseDF(companies)

In [3]:
# Get best country and top 3 cities in said country
country = mn.getTopCountry(base_df)
cities = mn.getTop3Cities(base_df, country)
main_df = mn.buildTopCitiesDF(base_df, country, cities)
main_df.head(5)
#showDF(df)

,Company,City,Country,lat,lon
365,Fandango,Los Angeles,USA,34.031646,-118.455352
512,Tubefilter,Los Angeles,USA,34.070436,-118.350486
838,Blip'd,Los Angeles,USA,34.076061,-118.372953
578,FanPrice,Los Angeles,USA,38.233786,-122.636071
584,SportsGenie,Los Angeles,USA,34.052200,-118.242900


In [4]:
# cities = ['New York', 'San Francisco', 'Los Angeles']
list_count_design_companies = []
for city in cities:
    num_design_companies = mn.getCompanies_TagCityCountry(c, '(design)', country, city)
    list_count_design_companies.append([city, num_design_companies])
df_cities = pd.DataFrame(list_count_design_companies, columns=['City', 'Design Companies'])
#showDF(df_cities)

In [5]:
for index, row in main_df.iterrows():
    # Design companies in city
    main_df = mn.buildDesignCompanies(index, row, main_df, df_cities, cities)
    
    # Distance to NIGHTCLUB
    distance = fs.getClosesVenue('Nightclub', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Nightclub')

    # Distance to ELEMENTARY SCHOOL
    distance = fs.getClosesVenue('Elementary School', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Elementary School')

    # Distance to MIDDLE SCHOOL
    distance = fs.getClosesVenue('Middle School', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Middle School')

    # Distance to HIGH SCHOOL
    distance = fs.getClosesVenue('High School', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'High School')

    # Distance to STARBUCKS
    distance = fs.getClosesVenue('starbucks', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Starbucks')

    # Distance to AIRPORT
    distance = fs.getClosesVenue('airport', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Airport')

    # Distance to TRAIN STATION 
    distance = fs.getClosesVenue('Train Station', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Train')

    # Distance to BASKETBALL COURT
    distance = fs.getClosesVenue('Basketball Court', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Basketball')

    # Distance to VEGAN RESTAURANT
    distance = fs.getClosesVenue('Vegetarian / Vegan Restaurant', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Vegan')

    # Distance to PET SERVICE
    distance = fs.getClosesVenue('Pet Service', [row.lon, row.lat])
    main_df = fs.addDistanceInfo(main_df, distance, index, 'Pet') 

    main_df = main_df.dropna()

c:\Users\franc\OneDrive\Escritorio\PROJECTS\IH_Project3\src\mongo.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df.loc[index, 'Design Companies'] = df_cities.loc[2, 'Design Companies']


KeyboardInterrupt: 

In [ ]:
main_df.to_csv ('..\data\mainDF.csv', index = None, header=True) 

## TESTING AREA ##

In [ ]:
distance = fs.getClosesVenue('Nightclub', [-118.455352, 34.031646])
print("Distance: ", distance['results'][0]['distance'])



Distance:  517


In [ ]:
main_df = main_df.dropna()
main_df

,Company,City,Country,lat,lon,Design Companies,Nightclub,Elementary School,Middle School,High School,Starbucks,Airport,Train,Basketball,Vegan,Pet
365,Fandango,Los Angeles,USA,34.031646,-118.455352,10.0,517.0,1481.0,621.0,1581.0,675.0,365.0,245.0,1004.0,1631.0,306.0
512,Tubefilter,Los Angeles,USA,34.070436,-118.350486,10.0,797.0,1520.0,2198.0,1605.0,640.0,919.0,354.0,521.0,1019.0,298.0
838,Blip'd,Los Angeles,USA,34.076061,-118.372953,10.0,387.0,1468.0,1950.0,1379.0,252.0,708.0,1503.0,1261.0,20.0,471.0
578,FanPrice,Los Angeles,USA,38.233786,-122.636071,10.0,348.0,955.0,800.0,1163.0,416.0,32.0,403.0,11972.0,5408.0,53.0
584,SportsGenie,Los Angeles,USA,34.052200,-118.242900,10.0,328.0,1710.0,1824.0,1330.0,130.0,292.0,420.0,1020.0,1157.0,494.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,SonicLiving,San Francisco,USA,37.844803,-122.290897,20.0,737.0,2750.0,368.0,1341.0,220.0,395.0,472.0,239.0,1221.0,1769.0
654,TubeSpree,San Francisco,USA,37.781386,-122.421406,20.0,394.0,681.0,191.0,277.0,494.0,573.0,621.0,644.0,299.0,499.0
362,eBaum's World,San Francisco,USA,37.786393,-122.398775,20.0,129.0,1813.0,658.0,2219.0,206.0,380.0,401.0,509.0,510.0,433.0
123,Outspark,San Francisco,USA,37.775196,-122.419204,20.0,121.0,255.0,255.0,221.0,225.0,270.0,8.0,799.0,280.0,301.0


In [ ]:
## FORMULA
# ponderated = dist * human * importance
# where:   dist => distance to venue
#          human => interested employees / total employees
#          importance => percent importance given
def valueLocation(row):
    design = row['Design Companies'] * (20/87) * 0.2
    club = row['Nightclub'] * (87/87) * 0.1
    eSchool = row['Elementary School'] * (26/87)
    mSchool = row['Middle School'] * (26/87)
    hSchool = row['High School'] * (26/87)
    school = (eSchool+mSchool+hSchool) * 0.25
    starbucks = row['Starbucks'] * (10/87) * 0.1
    airport = row['Airport'] * (20/87)
    train = row['Train'] * (20/87)
    transport = (airport+train) * 0.2
    basketball = row['Basketball'] * (1/87) * 0.05
    vegan = row['Vegan'] * (1/87) * 0.05
    pet = row['Pet'] * (1/87) * 0.05

    return design + club + school + starbucks + transport + basketball + vegan + pet

for index, row in main_df.iterrows():
    main_df.loc[index, 'Value'] = valueLocation(row)

C:\Users\franc\AppData\Local\Temp\ipykernel_15628\2341022614.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_df.loc[index, 'Value'] = full


In [ ]:
chosenOffice = main_df.iloc[0]

response = fs.getVenue('Outspark', [chosenOffice.lat, chosenOffice.lon])
response
# Get coordinates in needed format:
#       Using FourSquare -> Get office location
#                        -> Get near venues
# Create gdf
# Generate Map


AttributeError: module 'foursquare' has no attribute 'getVenue'